In [25]:
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))

Python: 2.7.13 |Anaconda 4.3.1 (64-bit)| (default, Dec 20 2016, 23:09:15) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
scipy: 0.18.1
numpy: 1.11.3
matplotlib: 2.0.0
pandas: 0.19.2
sklearn: 0.18.1


In [26]:
# Load libraries
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR

In [27]:
import datetime
from datetime import date,time
import time

In [28]:
pd.options.mode.chained_assignment = None 

In [29]:
#def time_num(date):
#    intervalo = 30
    
#    hora = date.hour
#    minuto = date.minute
    
#    num_minutes = hora*60 + minuto
    
#    time_bin = num_minutes / intervalo
#    hour_bin = num_minutes / 60
#    min_bin = (time_bin * intervalo) % 60
    
#    return ((hour_bin * 60 + min_bin + intervalo) / 2.0)/(60*24)

In [47]:
dataset = pd.read_csv('data/trip_train.csv')
weather = pd.read_csv('data/weather.csv')
station = pd.read_csv('data/station.csv')

In [48]:
station = station.loc[:,['id','lat','long']]

In [49]:
weather.date = pd.to_datetime(weather.date)

In [50]:
weather.precipitation_inches = weather.precipitation_inches.apply(lambda x: 0 if x == 'T' else x)
weather.precipitation_inches = pd.to_numeric(weather.precipitation_inches)

In [53]:
weather = weather.loc[:,['date','mean_temperature_f','precipitation_inches']].groupby('date').mean()
weather = pd.DataFrame(weather.to_records())

In [54]:
weather

,date,mean_temperature_f,precipitation_inches
0,2013-08-29,70.4,0.000
1,2013-08-30,73.0,0.000
2,2013-08-31,68.0,0.000
3,2013-09-01,70.0,0.000
4,2013-09-02,70.8,0.000
5,2013-09-03,68.8,0.342
6,2013-09-04,68.6,0.000
7,2013-09-05,68.0,0.000
8,2013-09-06,72.0,0.000
9,2013-09-07,76.2,0.000


In [55]:
dataset = dataset.loc[(dataset.duration < 86400) & (dataset.duration > 60),:]

In [37]:
prueba = dataset.head(400000)
prueba.describe()

,id,duration,start_station_id,end_station_id,bike_id
count,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000
mean,460428.359182,985.926872,57.866485,57.862005,427.635018
std,264685.723929,3254.075288,17.089099,17.177565,153.712385
min,4069.000000,61.000000,2.000000,2.000000,9.000000
25%,231089.750000,344.000000,50.000000,50.000000,334.000000
50%,459252.500000,517.000000,62.000000,62.000000,441.000000
75%,692916.250000,755.000000,70.000000,70.000000,546.000000
max,913459.000000,86363.000000,84.000000,84.000000,878.000000


In [38]:
prueba.start_date = pd.to_datetime(prueba.start_date)

In [39]:
prueba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400000 entries, 0 to 400210
Data columns (total 11 columns):
id                    400000 non-null int64
duration              400000 non-null int64
start_date            400000 non-null datetime64[ns]
start_station_name    400000 non-null object
start_station_id      400000 non-null int64
end_date              400000 non-null object
end_station_name      400000 non-null object
end_station_id        400000 non-null int64
bike_id               400000 non-null int64
subscription_type     400000 non-null object
zip_code              396121 non-null object
dtypes: datetime64[ns](1), int64(5), object(5)
memory usage: 36.6+ MB


In [56]:
prueba['date'] = prueba.start_date.apply(lambda x: datetime.datetime(x.year, x.month, x.day,0,0))
prueba['month'] = prueba.date.apply(lambda x: x.month)
prueba['year'] = prueba.date.apply(lambda x: x.year)

In [57]:
def devolver_estacion (fecha):
    if (fecha >= datetime.datetime(fecha.year,3,21)) & (fecha < datetime.datetime(fecha.year,6,21)):
        return 'Primavera'
    if (fecha >= datetime.datetime(fecha.year,6,21)) & (fecha < datetime.datetime(fecha.year,9,21)):
        return 'Verano'
    if (fecha >= datetime.datetime(fecha.year,9,21)) & (fecha < datetime.datetime(fecha.year,12,21)):
        return 'Otonio'
    else:
        return 'Invierno'

In [58]:
prueba['estacion'] = prueba.date.apply(lambda x: devolver_estacion(x))

In [59]:
prueba = pd.merge(prueba,weather,on='date',how='inner')

In [60]:
weather

,date,mean_temperature_f,precipitation_inches
0,2013-08-29,70.4,0.000
1,2013-08-30,73.0,0.000
2,2013-08-31,68.0,0.000
3,2013-09-01,70.0,0.000
4,2013-09-02,70.8,0.000
5,2013-09-03,68.8,0.342
6,2013-09-04,68.6,0.000
7,2013-09-05,68.0,0.000
8,2013-09-06,72.0,0.000
9,2013-09-07,76.2,0.000


In [61]:
prueba.describe()

,id,duration,start_station_id,end_station_id,bike_id,month,year,mean_temperature_f,precipitation_inches
count,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000,400000.000000
mean,460428.359182,985.926872,57.866485,57.862005,427.635018,6.471430,2014.213178,62.135443,0.011622
std,264685.723929,3254.075288,17.089099,17.177565,153.712385,3.276715,0.683852,6.761382,0.071511
min,4069.000000,61.000000,2.000000,2.000000,9.000000,1.000000,2013.000000,39.200000,0.000000
25%,231089.750000,344.000000,50.000000,50.000000,334.000000,4.000000,2014.000000,57.400000,0.000000
50%,459252.500000,517.000000,62.000000,62.000000,441.000000,7.000000,2014.000000,62.000000,0.000000
75%,692916.250000,755.000000,70.000000,70.000000,546.000000,9.000000,2015.000000,67.800000,0.000000
max,913459.000000,86363.000000,84.000000,84.000000,878.000000,12.000000,2015.000000,80.400000,1.982000


In [62]:
prueba['hour_start'] = prueba.start_date.apply(lambda x: x.hour)
prueba['weekend'] = prueba.start_date.apply(lambda x: 1 if x.weekday() >= 5 else 0)

In [63]:
prueba.subscription_type = prueba.subscription_type.apply(lambda x: 1 if x == 'Customer' else 0)

In [64]:
prueba = prueba.loc[:,['duration','hour_start','weekend','month','year','estacion','subscription_type','start_station_id','mean_temperature_f','precipitation_inches']]
prueba.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549678 entries, 0 to 549960
Data columns (total 11 columns):
id                    549678 non-null int64
duration              549678 non-null int64
start_date            549678 non-null object
start_station_name    549678 non-null object
start_station_id      549678 non-null int64
end_date              549678 non-null object
end_station_name      549678 non-null object
end_station_id        549678 non-null int64
bike_id               549678 non-null int64
subscription_type     549678 non-null object
zip_code              544300 non-null object
dtypes: int64(5), object(6)
memory usage: 50.3+ MB


In [65]:
prueba

,duration,hour_start,weekend,month,year,estacion,subscription_type,start_station_id,mean_temperature_f,precipitation_inches
0,396,8,0,8,2015,Verano,0,50,75.2,0.0
1,486,12,0,8,2015,Verano,0,54,75.2,0.0
2,1027,8,0,8,2015,Verano,0,69,75.2,0.0
3,354,8,0,8,2015,Verano,0,77,75.2,0.0
4,209,15,0,8,2015,Verano,0,69,75.2,0.0
5,849,8,0,8,2015,Verano,0,69,75.2,0.0
6,981,9,0,8,2015,Verano,0,50,75.2,0.0
7,239,13,0,8,2015,Verano,0,55,75.2,0.0
8,698,20,0,8,2015,Verano,1,51,75.2,0.0
9,414,17,0,8,2015,Verano,0,76,75.2,0.0


In [66]:
prueba['hora_pico'] = prueba.hour_start.apply(lambda x: 1 if (((x>=0) & (x<=3) ) | ((x>=10) & (x<=16))) else 0) 

In [67]:
estaciones = pd.get_dummies(prueba['estacion'])
prueba = prueba.join(estaciones)
prueba = prueba.drop('estacion',axis=1)
prueba = prueba.drop('hour_start',axis=1)
#prueba

In [68]:
meses = pd.get_dummies(prueba['month'])
prueba = pd.concat([prueba,meses],axis=1)
prueba = prueba.drop('month',axis=1)

In [69]:
anios = pd.get_dummies(prueba['year'])
prueba = pd.concat([prueba,anios],axis=1)
prueba = prueba.drop('year',axis=1)

In [70]:
prueba

,duration,weekend,subscription_type,start_station_id,mean_temperature_f,precipitation_inches,hora_pico,Invierno,Otonio,Primavera,...,6,7,8,9,10,11,12,2013,2014,2015
0,396,0,0,50,75.2,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,486,0,0,54,75.2,0.0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,1027,0,0,69,75.2,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,354,0,0,77,75.2,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,209,0,0,69,75.2,0.0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
5,849,0,0,69,75.2,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
6,981,0,0,50,75.2,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
7,239,0,0,55,75.2,0.0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
8,698,0,1,51,75.2,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
9,414,0,0,76,75.2,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [71]:
columnas = prueba.columns.tolist()
columnas = columnas[1:len(columnas)]
columnas.append('duration')
prueba = prueba[columnas]
len(prueba.columns.tolist())

26

In [72]:
from sklearn import preprocessing

In [103]:
from sklearn.preprocessing import MinMaxScaler

In [104]:
array = prueba.values
X = array[:,0:len(prueba.columns.tolist())-1]
Y = array[:,len(prueba.columns.tolist())-1]
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(rescaledX, Y, test_size=validation_size, random_state=seed)

# Pruebo los distintos algoritmos

In [105]:
start_time = time.time()
models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('CART', DecisionTreeRegressor()))
models.append(('RDMF',RandomForestRegressor()))
models.append(('EXTR',ExtraTreesRegressor()))
models.append(('ADBS',AdaBoostRegressor()))
models.append(('GDBR',GradientBoostingRegressor()))
#models.append(('NB', GaussianNB()))
#models.append(('SGD',SGDRegressor()))
#models.append(('SVM', SVR(kernel='linear')))
# evaluate each model in turn
for name, model in models:
    model.fit(X_train,Y_train)
    predictions = model.predict(X_validation)
    print "%s -> Mean Squared Error: %f" % (name, mean_squared_error(Y_validation, predictions))
    print model.score(X_validation,Y_validation)
print("--- %s seconds ---" % (time.time() - start_time))

RDMF -> Mean Squared Error: 10704603.879970
-0.0168643654721
EXTR -> Mean Squared Error: 11285079.078798
-0.0720055506431
ADBS -> Mean Squared Error: 41564372.988479
-2.94833197371
GDBR -> Mean Squared Error: 9364414.090912
0.110444524679
--- 229.198658943 seconds ---


# Pruebo ExtraTrees con distinto # de arboles

In [ ]:
start_time = time.time()
for k in range(1,11):
    lda = KNeighborsClassifier(n_neighbors = k)
    lda.fit(X_train, Y_train)
    predictions = lda.predict(X_validation)
    print "Con K = %i: %f" % (k,mean_squared_error(Y_validation, predictions))    
print("--- %s seconds ---" % (time.time() - start_time))

# Pruebo RandomForests con distinto # de arboles

In [ ]:
start_time = time.time()
for k in range(10,20):
    rdf = RandomForestRegressor(n_estimators = k)
    rdf.fit(X_train, Y_train)
    predictions = rdf.predict(X_validation)
    print "Con %i arboles: %f" % (k,mean_squared_error(Y_validation, predictions))    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
%matplotlib inline
rdf = RandomForestRegressor(n_estimators = 12)
rdf.fit(X_train, Y_train)
importances = rdf.feature_importances_
std = numpy.std([tree.feature_importances_ for tree in rdf.estimators_],
             axis=0)
indices = numpy.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X_train.shape[1]), indices)
plt.xlim([-1, X_train.shape[1]])
plt.show()

# ----------------------------------------------------------------------------------------------------

In [ ]:
#from sklearn.preprocessing import MinMaxScaler

In [ ]:
#scaler_train = MinMaxScaler(feature_range=(0, 1))
#rescaledX_train = scaler_train.fit_transform(X_train)

#scaler_validation = MinMaxScaler(feature_range=(0, 1))
#rescaledX_validation = scaler_validation.fit_transform(X_validation)

#normalknn = LogisticRegression()
#normalknn.fit(rescaledX_train, Y_train)
#predictions = normalknn.predict(rescaledX_validation)

#print mean_squared_error(Y_validation, predictions)

In [106]:
test = pd.read_csv('data/trip_test.csv')

In [107]:
test.start_date = pd.to_datetime(test.start_date)

In [108]:
test['date'] = test.start_date.apply(lambda x: datetime.datetime(x.year,x.month,x.day,0,0))

In [109]:
test['weekend'] = test.start_date.apply(lambda x: 1 if x.weekday() >= 5 else 0)
test['month'] = test.date.apply(lambda x: x.month)
test['year'] = test.date.apply(lambda x: x.year)


In [110]:
test.subscription_type = test.subscription_type.apply(lambda x: 1 if x == 'Customer' else 0)

In [111]:
test = pd.merge(test,weather,on='date',how='inner')

In [112]:
test['hour_start'] = test.start_date.apply(lambda x: x.hour)

In [113]:
test['estacion'] = test.date.apply(lambda x: devolver_estacion(x))

In [114]:
test['hora_pico'] = test.hour_start.apply(lambda x: 1 if (((x>=0) & (x<=3) ) | ((x>=10) & (x<=16))) else 0)

In [115]:
test = test.loc[:,['weekend','subscription_type','start_station_id','mean_temperature_f','precipitation_inches','hora_pico','year','month','estacion']]

In [116]:
test

,weekend,subscription_type,start_station_id,mean_temperature_f,precipitation_inches,hora_pico,year,month,estacion
0,1,1,60,68.4,0.000,1,2014,10,Otonio
1,1,1,45,68.4,0.000,0,2014,10,Otonio
2,1,1,64,68.4,0.000,1,2014,10,Otonio
3,1,0,70,68.4,0.000,0,2014,10,Otonio
4,1,0,39,68.4,0.000,1,2014,10,Otonio
5,1,0,51,68.4,0.000,0,2014,10,Otonio
6,1,0,4,68.4,0.000,0,2014,10,Otonio
7,1,1,47,68.4,0.000,1,2014,10,Otonio
8,1,1,76,68.4,0.000,1,2014,10,Otonio
9,1,0,60,68.4,0.000,0,2014,10,Otonio


In [94]:
gdb = GradientBoostingRegressor()
gdb.fit(X,Y)
predictions = gdb.predict(array_test)

In [ ]:
#test = test.drop('start_date',axis=1)
test

In [86]:
estaciones = pd.get_dummies(test['estacion'])
test = test.join(estaciones)
test = test.drop('estacion',axis=1)
#test = test.drop('hour_start',axis=1)

In [87]:
meses = pd.get_dummies(test['month'])
test = pd.concat([test,meses],axis=1)
test = test.drop('month',axis=1)

In [88]:
anios = pd.get_dummies(test['year'])
test = pd.concat([test,anios],axis=1)
test = test.drop('year',axis=1)

In [89]:
#test = pd.merge(test,weather,on='date',how='inner')

In [90]:
#test = test.drop('date',axis=1)

In [91]:
#test = test[['mean_temperature_f','time_num','weekend']]

In [92]:
test

,weekend,subscription_type,start_station_id,mean_temperature_f,precipitation_inches,hora_pico,Invierno,Otonio,Primavera,Verano,...,6,7,8,9,10,11,12,2013,2014,2015
0,1,1,60,68.4,0.000,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1,1,1,45,68.4,0.000,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
2,1,1,64,68.4,0.000,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
3,1,0,70,68.4,0.000,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
4,1,0,39,68.4,0.000,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
5,1,0,51,68.4,0.000,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
6,1,0,4,68.4,0.000,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
7,1,1,47,68.4,0.000,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
8,1,1,76,68.4,0.000,1,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
9,1,0,60,68.4,0.000,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0


In [117]:
array_test = test.values
rescaledX_test = scaler.fit_transform(array_test)
validation_size = 0.20

ValueError: could not convert string to float: Invierno

In [ ]:
rfr = RandomForestRegressor(n_estimators = 11)
rfr.fit(X, Y)
predictions = rfr.predict(array_test)

In [95]:
predictions = pd.DataFrame(predictions)

In [96]:
predictions.columns = ['duration']
predictions

,duration
0,3081.459141
1,2709.809999
2,2960.241342
3,809.472617
4,674.411160
5,645.367548
6,581.527058
7,3593.235598
8,3959.811110
9,650.903101


In [97]:
test = pd.read_csv('data/trip_test.csv')

In [98]:
test = test.join(predictions)

In [99]:
test = test.loc[:,['id','duration']]

In [ ]:
test.to_csv('submission20.csv',index=False)

In [100]:
respuesta = pd.read_csv('data/trip.csv')

In [101]:
respuesta = pd.merge(test,respuesta,on='id',how='inner')

In [102]:
mean_squared_error(respuesta['duration_y'],respuesta['duration_x'])

43899094.869477645

In [ ]:
test.describe()